<a href="https://colab.research.google.com/github/spacewal/Module-3/blob/main/Copy_of_Module_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### Alberto Diaz
### Laura Cepero Navarro
### Orialys Ruiz

In [ ]:
!pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00


In [ ]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)
# data:  https://github.com/fenago/datasets/raw/main/Online%20Retail.xlsx

import lifetimes
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter # BG/NBD
from lifetimes import GammaGammaFitter # Gamma-Gamma Model
from lifetimes.plotting import plot_frequency_recency_matrix
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/spacewal/Module-3/main/synthetic_grocery_data.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   User Name         873 non-null    object 
 1   User ID           873 non-null    object 
 2   User DOB          873 non-null    object 
 3   User Email        873 non-null    object 
 4   Transaction Date  873 non-null    object 
 5   Transaction ID    873 non-null    object 
 6   Category          873 non-null    object 
 7   Product           873 non-null    object 
 8   Price             873 non-null    float64
 9   Volume Purchased  873 non-null    int64  
 10  Total Price       873 non-null    float64
dtypes: float64(2), int64(1), object(8)
memory usage: 75.1+ KB


### Below we channged the column "Transaction Date" from object to datetime

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   User Name         873 non-null    object        
 1   User ID           873 non-null    object        
 2   User DOB          873 non-null    object        
 3   User Email        873 non-null    object        
 4   Transaction Date  873 non-null    datetime64[ns]
 5   Transaction ID    873 non-null    object        
 6   Category          873 non-null    object        
 7   Product           873 non-null    object        
 8   Price             873 non-null    float64       
 9   Volume Purchased  873 non-null    int64         
 10  Total Price       873 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 75.1+ KB


In [ ]:
df.describe()

,Price,Volume Purchased,Total Price
count,873.000000,873.000000,873.000000
mean,2.855613,5.607102,16.152806
std,2.557375,2.866333,18.638378
min,0.500000,1.000000,0.700000
25%,1.300000,3.000000,5.000000
50%,2.000000,6.000000,10.000000
75%,3.000000,8.000000,19.800000
max,12.000000,10.000000,120.000000


In [ ]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

### We found an outlier in "Total Price" and we decided to cap that column.

In [ ]:
capping_outliers(df,'Total Price')

In [ ]:
df.describe()

,Price,Volume Purchased,Total Price
count,873.000000,873.000000,873.000000
mean,2.855613,5.607102,14.634479
std,2.557375,2.866333,13.081797
min,0.500000,1.000000,1.800000
25%,1.300000,3.000000,5.000000
50%,2.000000,6.000000,10.000000
75%,3.000000,8.000000,19.800000
max,12.000000,10.000000,49.400000


# RFM

### We decided to look at the max and min on the Transaction Date column to see what our observation period should be.

In [ ]:
df.head(2)

,User Name,User ID,User DOB,User Email,Transaction Date,Transaction ID,Category,Product,Price,Volume Purchased,Total Price
0,Norma Fisher,7c65c1e5-82e2-4662-b728-b4fa42485e3a,1955-02-12,tammy59@example.org,2024-02-12 06:13:31.672365,1846d424-c17c-4279-a3c6-612f48268673,Meat,Fish,6.0,4,24.0
1,Danielle Browning,9a164106-cf6a-459e-b486-2b21fb97d435,2012-10-17,corey15@example.com,2024-02-13 11:44:31.656054,9c6316b9-50f2-4455-af25-e2a25a921187,Grains,Bread,1.5,10,15.0


In [ ]:
df['Total Price'] = df['Price'] * df['Volume Purchased']

In [ ]:
df['Transaction Date'].max()

Timestamp('2024-02-23 06:20:45.207358')

In [ ]:
df['Transaction Date'].min()

Timestamp('2024-02-01 00:11:52.891986')

In [ ]:
clv = lifetimes.utils.summary_data_from_transaction_data(df,'User ID','Transaction Date','Total Price',observation_period_end='2024-02-23 06:20:45.207358')

### This is our first CLV without refining it, because we have seen customer with no RFM value. We decided to refine the CLV for customer who have made more than one purchase.

In [ ]:
clv.head(10)

,frequency,recency,T,monetary_value
User ID,,,,
033c1469-4f06-4a18-b4a2-64447d3c279b,0.0,0.0,12.0,0.000000
03983ca8-ea7e-4d49-8c77-8ea6eb2083e6,1.0,6.0,15.0,32.200000
08135d58-6a16-49ad-9fe1-b30791725f0a,0.0,0.0,19.0,0.000000
0876daaa-01ce-4fc6-b2d1-16fe8f08963c,0.0,0.0,10.0,0.000000
0a8aa1e4-5c7c-4c62-81a4-f7a165264961,1.0,6.0,16.0,25.000000
0d18533e-7efa-4faf-98ba-0bf3675e0c52,0.0,0.0,5.0,0.000000
0fb2de1e-a0b9-46c2-bdb9-31ce05a0f421,0.0,0.0,21.0,0.000000
10e857f8-8986-448f-8b3e-00e76151787e,1.0,13.0,22.0,77.200000
13bfa9fe-9cdd-4f6c-b0cd-c4a836a5be06,0.0,0.0,12.0,0.000000


In [ ]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
User ID,,,,
1734a26c-92e9-4e89-889b-30a0809f2923,3.0,21.0,22.0,45.066667
211e55c3-9cc7-43ad-a264-190f486c0adf,3.0,15.0,20.0,56.866667
2446ecee-caa0-4679-ae83-a7b2c857d8c3,4.0,14.0,15.0,36.650000
25cc2478-276d-4e19-b924-e06aaef5baef,2.0,12.0,21.0,45.600000
2b0c9fdd-b61b-43e9-8b60-fefcc47e330e,2.0,20.0,22.0,37.000000


# BG/NBD Model

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)  # This will allow you to predict FUTURE purchase by taking into account F, R, T  penalizer is used to avoid overfitting
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 52 subjects, a: 0.00, alpha: 46.66, b: 0.00, r: 7.35>

### We are trying to predict customers who have made more than one purchase and predict what their expected purchase would be in six months.For example, cutomer 2 is expected to purchase half of their monetary value. Meanwhile cutomer 5 will purchase more than double their monetary value.

In [ ]:
t = 180 # 6 month/180 day period
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
clv.sort_values(by='expected_purc_6_months',ascending=False).head(5)

,frequency,recency,T,monetary_value,expected_purc_6_months
User ID,,,,,
fd723837-705b-4cc2-a40c-a20c702a5b67,4.0,10.0,13.0,33.062500,34.232159
2d492ca2-41f4-417d-b514-1524e08c28eb,4.0,8.0,14.0,64.925000,33.667866
2446ecee-caa0-4679-ae83-a7b2c857d8c3,4.0,14.0,15.0,36.650000,33.121875
4c88b9d8-ab12-4b53-8f42-cebe23b870f3,4.0,14.0,15.0,33.700000,33.121875
95919900-5e7c-4aad-b3df-770aad0cabfc,3.0,8.0,11.0,12.333333,32.297917


# Gamma-Gamma Model

### The correlation between 'frequency' and 'monetary_value' is approximately -0.146303. This suggests a slight negative linear relationship, meaning that as the frequency of purchases increases, the average monetary value of those purchases tends to decrease slightly. However, the correlation is relatively weak, implying that the relationship is not strongly linear and that other factors may also influence the monetary value.






In [ ]:
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,-0.146303
monetary_value,-0.146303,1.000000


In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(clv["frequency"],
        clv["monetary_value"])

<lifetimes.GammaGammaFitter: fitted with 52 subjects, p: 4.18, q: 0.78, v: 4.03>

In [ ]:
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
User ID,,,,,,
c48ae374-06dc-4579-a476-40a1b6a659ef,2.0,11.0,14.0,145.075000,27.733517,4047.454132
57030ede-889c-4ae2-8e3c-0f2d4332559d,3.0,9.0,14.0,89.900000,30.700691,2754.387721
f33c4233-c833-407e-b4d5-6ffcdf4a518a,3.0,10.0,19.0,97.333333,28.362972,2751.956543
33a1d1c2-ad4a-4155-809f-cd8f739cd488,4.0,20.0,20.0,85.875000,30.637626,2605.649472
675d0855-2d55-40c9-8576-598359cd012a,2.0,18.0,21.0,102.075000,24.864409,2567.929553


In [ ]:
clv['Segment'] =  pd.qcut(clv['six_Months_CLV'],4,labels = ['Hibernating','Need Attention', 'LoyalCustomers', 'Champions'])

### Through a qcut we were able to group together cutomers that 'Hibernating','Need Attention', 'LoyalCustomers', 'Champions' and this indicates which cutomers are worth pursuing and which are not.

In [ ]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV,Segment
User ID,,,,,,,
1734a26c-92e9-4e89-889b-30a0809f2923,3.0,21.0,22.0,45.066667,27.123761,1237.777374,Need Attention
211e55c3-9cc7-43ad-a264-190f486c0adf,3.0,15.0,20.0,56.866667,27.937509,1599.057803,LoyalCustomers
2446ecee-caa0-4679-ae83-a7b2c857d8c3,4.0,14.0,15.0,36.650000,33.121875,1220.957654,Need Attention
25cc2478-276d-4e19-b924-e06aaef5baef,2.0,12.0,21.0,45.600000,24.864409,1174.698526,Need Attention
2b0c9fdd-b61b-43e9-8b60-fefcc47e330e,2.0,20.0,22.0,37.000000,24.502291,948.519496,Hibernating


### Here we are continuting the segemet ctomers by group and it its taking the average per each group column. It is very clear that champions and loyal cutomers are going to spen more than cutomers that Need Attention and Hibernating.

In [ ]:
clv.groupby('Segment').mean()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
Segment,,,,,,
Hibernating,2.769231,12.923077,19.076923,22.255128,27.777530,647.211133
Need Attention,2.846154,14.461538,17.846154,41.542628,28.543502,1196.488631
LoyalCustomers,3.153846,15.615385,19.769231,56.792949,28.461972,1625.078054
Champions,2.769231,12.615385,17.538462,89.689744,28.435453,2540.860278
